# ***Experiment 4 (Function Name - Raw)***

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***Data Preprocessing***

In [2]:
import pandas as pd
path = '/content/drive/MyDrive/Ashik/Thesis/'
df_category = pd.read_csv(f'{path}data/collections_final.csv')

In [3]:
contracts_with_transactions =  df_category['category'].unique()
contracts_with_transactions = contracts_with_transactions.tolist()
contracts_with_transactions

['arts', 'avatar', 'collectibles', 'memberships', 'games']

In [4]:
df_category['category'].value_counts()

category
avatar          601
arts            600
games           431
memberships     409
collectibles    365
Name: count, dtype: int64

In [5]:
days = 0
df_method = pd.read_csv(f'{path}/data/raw_data_first_{days}_final.csv')
df_method.head()

,contract_address,block_date,txn_hash,value,min_block_date,cutoff_block_date,method
0,0x099a16f0414cb0cc0555d5f1f8140166462d39ff,2021-10-01,BD2B0AA77BD47F6A9FEDC004CF17E3312112E114567E9E...,0.000000e+00,2021-10-01,2021-10-01,0xa22cb465
1,0x099a16f0414cb0cc0555d5f1f8140166462d39ff,2021-10-01,94D6FD95DC45026998BA5BD804DC9113001340CEA279E0...,0.000000e+00,2021-10-01,2021-10-01,0xa22cb465
2,0x099a16f0414cb0cc0555d5f1f8140166462d39ff,2021-10-01,F7D00DCFA07BC5B68ADD3607A5C65687C7CDCF0B3B9754...,0.000000e+00,2021-10-01,2021-10-01,0xa22cb465
3,0x099a16f0414cb0cc0555d5f1f8140166462d39ff,2021-10-01,DDBD3F8175FC494F0EFDD6A947CA62DCFE0E60D22B2F2C...,3.000000e+16,2021-10-01,2021-10-01,0xbf3f2153
4,0x099a16f0414cb0cc0555d5f1f8140166462d39ff,2021-10-01,81768E048E7386236C309C0EA62833839EA6128FC284DB...,0.000000e+00,2021-10-01,2021-10-01,0xa22cb465


In [6]:
print("before merge ::: ", df_method.shape)
df_method = pd.merge(df_method, df_category, on=['contract_address'], how='inner')
print("after merge ::: ", df_method.shape)

before merge :::  (5321062, 7)
after merge :::  (5397819, 12)


In [7]:
contracts_with_transactions = set(df_method['contract_address'])

In [8]:
print(f"Lenght of contracts_with_transactions {len(contracts_with_transactions)}")
df_method.drop(columns = ['category'], inplace=True)
print()

print('df_category shape ::: ', df_category.shape)
df_category = df_category[df_category['contract_address'].isin(contracts_with_transactions)]
print('df_category shape ::: ', df_category.shape)
print()

print(df_category['category'].value_counts())

print()
categories_of_interest = list(set(df_category['category']))
print(f"categories_of_interest : {categories_of_interest}")

Lenght of contracts_with_transactions 2375

df_category shape :::  (2406, 6)
df_category shape :::  (2406, 6)

category
avatar          601
arts            600
games           431
memberships     409
collectibles    365
Name: count, dtype: int64

categories_of_interest : ['memberships', 'collectibles', 'avatar', 'games', 'arts']


In [9]:
df_functions = pd.read_csv(f'{path}data/function_name.csv')
df_functions.shape

(2806, 4)

In [10]:
print()
df_txn = pd.merge(df_method, df_functions, on='method', how='left' )
print(f"df_txn : {df_txn.head()}")

print()
df_txn['function'] = df_txn['function'].apply(lambda x: str(x).split("(")[0])

print(f"df_txn : {df_txn.head()}")

print()
print(f" method n unique : {df_txn['method'].nunique()}, function n unique : {df_txn['function'].nunique()} ")

print()
df_txn[df_txn['function'].isnull()].shape[0], df_txn[df_txn['function'] == 'nan'].shape[0]

print()
print(f"df_category : {df_category.head()}")

print()
print(f"df_category_shape : {df_category.shape}")



df_txn :                              contract_address  block_date  \
0  0x099a16f0414cb0cc0555d5f1f8140166462d39ff  2021-10-01   
1  0x099a16f0414cb0cc0555d5f1f8140166462d39ff  2021-10-01   
2  0x099a16f0414cb0cc0555d5f1f8140166462d39ff  2021-10-01   
3  0x099a16f0414cb0cc0555d5f1f8140166462d39ff  2021-10-01   
4  0x099a16f0414cb0cc0555d5f1f8140166462d39ff  2021-10-01   

                                            txn_hash  value min_block_date  \
0  BD2B0AA77BD47F6A9FEDC004CF17E3312112E114567E9E...    0.0     2021-10-01   
1  BD2B0AA77BD47F6A9FEDC004CF17E3312112E114567E9E...    0.0     2021-10-01   
2  94D6FD95DC45026998BA5BD804DC9113001340CEA279E0...    0.0     2021-10-01   
3  94D6FD95DC45026998BA5BD804DC9113001340CEA279E0...    0.0     2021-10-01   
4  F7D00DCFA07BC5B68ADD3607A5C65687C7CDCF0B3B9754...    0.0     2021-10-01   

  cutoff_block_date      method                                           url  \
0        2021-10-01  0xa22cb465  https://opensea.io/collection/unstackedt

# ***Feature Engineering***

In [11]:
contract_address = df_category['contract_address'].values.tolist()
print(f"contract_address:  {len(contract_address)}")
print()
print(f"df_method n unique : {df_method['contract_address'].nunique()}")
print()
txn_rows = df_txn[df_txn['contract_address'].isin(contract_address)].shape[0]
print()

df_txn = df_txn[df_txn['contract_address'].isin(contract_address)]
print(f"df_txn : {df_txn.shape}")

## Feature Engineering
df_method = df_txn.groupby(['contract_address', 'function']).agg({'txn_hash' : 'nunique'})
df_method = df_method.pivot_table(index=['contract_address'], columns = 'function', values = 'txn_hash')

print(f"df_method : {df_method.shape}, val_df_method : {df_method.shape} ")

df_feat_engg = df_method

df_feat_engg['contract_address'] = df_method.index.values
df_feat_engg.reset_index(drop=True, inplace=True)

print()

print(df_feat_engg.shape)
df_feat_engg = pd.merge(df_feat_engg, df_category, on = 'contract_address', how='inner')

print()
print(df_feat_engg.shape)

print(f"df_feat_engg : {df_feat_engg.head()}")
print()
print(f"df_feat_engg : {df_feat_engg.describe()}")

contract_address:  2406

df_method n unique : 2375


df_txn : (6709545, 14)
df_method : (2375, 2251), val_df_method : (2375, 2251) 

(2375, 2252)

(2406, 2257)
df_feat_engg :    AaANwg8  AcoraidaMonicaWantsToKeepALogOfTheWinner  Activate  AddFish  \
0      NaN                                       NaN       NaN      NaN   
1      NaN                                       NaN       NaN      NaN   
2      NaN                                       NaN       NaN      NaN   
3      NaN                                       NaN       NaN      NaN   
4      NaN                                       NaN       NaN      NaN   

   AddModerator  AddNewBooster  AddNewCard  AddNewChip  Airdrop  Attack  ...  \
0           NaN            NaN         NaN         NaN      NaN     NaN  ...   
1           NaN            NaN         NaN         NaN      NaN     NaN  ...   
2           NaN            NaN         NaN         NaN      NaN     NaN  ...   
3           NaN            NaN         NaN         NaN

# ***LightGBM***

Proposed Model

In [12]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score
import pickle
import os

le = LabelEncoder()
df_feat_engg['category']= le.fit_transform(df_feat_engg['category'])

print(df_feat_engg.dtypes)

X = df_feat_engg.drop(labels=['collection', 'category','description','contract_address','url','blockchain'], axis=1)
y = df_feat_engg['category'].values.astype('int32')

param = {'objective': 'multiclass',
         'boosting': 'gbdt',
         'learning_rate': 0.1,
         'num_iterations': 5000,
         'min_data_in_leaf': 10,
         'bagging_fraction': 0.80,
         'bagging_freq': 5,
         'feature_fraction': 0.8,
         }

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics_list = []

for i, (train_index, test_index) in enumerate(fold.split(X, y)):
    print(i)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print(len(X_train),len(X_test),len(y_train),len(y_test))

    model = LGBMClassifier(**param)

    es = early_stopping(stopping_rounds=200, first_metric_only= True)
    model = model.fit(X_train,y_train,
                    callbacks = [es, log_evaluation(100)],
                    eval_set = (X_test, y_test),
                    eval_metric = 'multiclass'
                    )
    print()
    print(f" model.classes_  : {model.classes_}")

    predict = model.predict_proba(X_test)
    print(predict.shape)

    X_test['predicted_class'] = [np.argmax(i) for i in predict]
    y_test_array = np.array(y_test)

    X_test['is_correct'] = np.where(y_test_array ==  X_test['predicted_class'], 1, 0)

    is_correct = X_test['is_correct'].sum()
    print(f"is_correct      : {X_test['is_correct'].sum()}")

    roc_auc = roc_auc_score(y_test_array, predict, average='macro', multi_class='ovr')
    f1 = f1_score(y_test_array, X_test['predicted_class'], average='micro')
    recall = recall_score(y_test_array, X_test['predicted_class'], average='micro')
    precision = precision_score(y_test_array, X_test['predicted_class'], average='micro')

    print(f"roc_auc_score    : {roc_auc}")
    print(f"f1_score         : {f1}")
    print(f"recall_score     : {recall}")
    print(f"precision_score  : {precision}")

    metrics_dict = {
        'tot_samples': len(y_test_array),
        'is_correct': is_correct,
        'roc_auc_score': roc_auc,
        'f1_score': f1,
        'recall_score': recall,
        'precision_score': precision
    }

    # directory = "/dbfs/FileStore/SCCC/3_day_models"
    # if not os.path.exists(directory):
    #     os.makedirs(directory)

    metrics_list.append(metrics_dict)

    # file_path = f"{directory}/model_{i}.pkl"

    # if os.path.exists(file_path):
    #  os.remove(file_path)

    # with open(file_path, 'wb') as file:
    #     pickle.dump(model, file)


AaANwg8                                     float64
AcoraidaMonicaWantsToKeepALogOfTheWinner    float64
Activate                                    float64
AddFish                                     float64
AddModerator                                float64
                                             ...   
url                                          object
collection                                   object
description                                  object
category                                      int64
blockchain                                   object
Length: 2257, dtype: object
0
1924 482 1924 482
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] baggi

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001607 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2464
[LightGBM] [Info] Number of data points in the train set: 1924, number of used features: 101
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM]

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 wil

...

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001674 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2438
[LightGBM] [Info] Number of data points in the train set: 1925, number of used features: 99
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 wil

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 wil

In [13]:
results_lgbm = pd.DataFrame(metrics_list)
results_lgbm

,tot_samples,is_correct,roc_auc_score,f1_score,recall_score,precision_score
0,482,424,0.989976,0.879668,0.879668,0.879668
1,481,438,0.988668,0.910603,0.910603,0.910603
2,481,420,0.985501,0.873181,0.873181,0.873181
3,481,432,0.989120,0.898129,0.898129,0.898129
4,481,419,0.986605,0.871102,0.871102,0.871102


In [14]:
results_lgbm.to_csv(f'{path}results/exp-4-lgbm.csv',index = False)

In [15]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score
import pickle
import os

le = LabelEncoder()
df_feat_engg['category']= le.fit_transform(df_feat_engg['category'])

print(df_feat_engg.dtypes)

X = df_feat_engg.drop(labels=['collection', 'category','description','contract_address','url','blockchain'], axis=1)
y = df_feat_engg['category'].values.astype('int32')

param = {'objective': 'multiclass',
         'boosting': 'gbdt',
         'learning_rate': 0.1,
         'num_iterations': 5000,
         'min_data_in_leaf': 10,
         'bagging_fraction': 0.80,
         'bagging_freq': 5,
         'feature_fraction': 0.8,
         }

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics_list = []
metrics_list1 = []

for i, (train_index, test_index) in enumerate(fold.split(X, y)):
    print(i)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print(len(X_train),len(X_test),len(y_train),len(y_test))

    model = LGBMClassifier(**param)

    es = early_stopping(stopping_rounds=200, first_metric_only= True)
    model = model.fit(X_train,y_train,
                    callbacks = [es, log_evaluation(100)],
                    eval_set = (X_test, y_test),
                    eval_metric = 'multiclass'
                    )
    print()
    print(f" model.classes_  : {model.classes_}")

    predict = model.predict_proba(X_test)
    print(predict.shape)

    X_test['predicted_class'] = [np.argmax(i) for i in predict]
    y_test_array = np.array(y_test)

    X_test['is_correct'] = np.where(y_test_array ==  X_test['predicted_class'], 1, 0)

    is_correct = X_test['is_correct'].sum()
    print(f"is_correct      : {X_test['is_correct'].sum()}")

    roc_auc = roc_auc_score(y_test_array, predict, average='macro', multi_class='ovr')
    f1 = f1_score(y_test_array, X_test['predicted_class'], average='micro')
    recall = recall_score(y_test_array, X_test['predicted_class'], average='micro')
    precision = precision_score(y_test_array, X_test['predicted_class'], average='micro')

    print(f"roc_auc_score    : {roc_auc}")
    print(f"f1_score         : {f1}")
    print(f"recall_score     : {recall}")
    print(f"precision_score  : {precision}")

    metrics_dict = {
        'tot_samples': len(y_test_array),
        'is_correct': is_correct,
        'roc_auc_score': roc_auc,
        'f1_score': f1,
        'recall_score': recall,
        'precision_score': precision
    }

    predicted_classes = [np.argmax(pred) for pred in predict]

    f1_scores = []
    for category_index in range(len(le.classes_)):
        true_labels = (y_test == category_index)
        predicted_labels = (predicted_classes == category_index)
        category_f1 = f1_score(true_labels, predicted_labels)
        f1_scores.append(category_f1)

    metrics_dict1 = {
        'tot_samples': len(y_test),
        'roc_auc_score': roc_auc,
        'category_f1_scores': f1_scores,
    }

    metrics_list1.append(metrics_dict1)

    # directory = "/dbfs/FileStore/SCCC/3_day_models"
    # if not os.path.exists(directory):
    #     os.makedirs(directory)

    metrics_list.append(metrics_dict1)

    # file_path = f"{directory}/model_{i}.pkl"

    # if os.path.exists(file_path):
    #  os.remove(file_path)

    # with open(file_path, 'wb') as file:
    #     pickle.dump(model, file)


AaANwg8                                     float64
AcoraidaMonicaWantsToKeepALogOfTheWinner    float64
Activate                                    float64
AddFish                                     float64
AddModerator                                float64
                                             ...   
url                                          object
collection                                   object
description                                  object
category                                      int64
blockchain                                   object
Length: 2257, dtype: object
0
1924 482 1924 482
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] baggi

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002257 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2464
[LightGBM] [Info] Number of data points in the train set: 1924, number of used features: 101
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM]

TypeError: Expected sequence or array-like, got <class 'bool'>

In [ ]:
predicted_labels

In [ ]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score
from itertools import combinations

le = LabelEncoder()
df_feat_engg['category'] = le.fit_transform(df_feat_engg['category'])

X = df_feat_engg.drop(labels=['collection', 'category','description','contract_address','url','blockchain'], axis=1)
y = df_feat_engg['category'].values.astype('int32')

param = {'objective': 'multiclass',
         'boosting': 'gbdt',
         'learning_rate': 0.1,
         'num_iterations': 5000,
         'min_data_in_leaf': 10,
         'bagging_fraction': 0.80,
         'bagging_freq': 5,
         'feature_fraction': 0.8,
         }

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics_list = []
roc_auc_scores = []

for i, (train_index, test_index) in enumerate(fold.split(X, y)):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = LGBMClassifier(**param)

    model = model.fit(X_train, y_train)

    predict = model.predict_proba(X_test)

    roc_auc = roc_auc_score(y_test, predict, average='macro', multi_class='ovr')
    roc_auc_scores.append(roc_auc)

    predicted_classes = [np.argmax(pred) for pred in predict]  # Convert probabilities to class labels

    # Calculate F1 score for each category
    f1_scores = []
    for category_index in range(len(le.classes_)):
        true_labels = (y_test == category_index)
        predicted_labels = (predicted_classes == category_index)
        category_f1 = f1_score(true_labels, predicted_labels)
        f1_scores.append(category_f1)

    metrics_dict = {
        'tot_samples': len(y_test),
        'roc_auc_score': roc_auc,
        'category_f1_scores': f1_scores,
    }

    metrics_list.append(metrics_dict)



In [ ]:
roc_auc_scores

In [ ]:
best_model_index = np.argmax(roc_auc_scores)
best_model_index

In [ ]:
metrics_list

In [ ]:
# Select the model with the highest ROC AUC score
best_model_index = np.argmax(roc_auc_scores)
best_model_metrics = metrics_list[best_model_index]

# Calculate pairwise F1-score for the predicted labels
predicted_labels = [metrics['category_f1_scores'] for metrics in metrics_list]
pairwise_f1_scores = []
for pair in combinations(predicted_labels, 2):
    pairwise_f1 = np.mean([f1_score(pair[0][i], pair[1][i]) for i in range(len(le.classes_))])
    pairwise_f1_scores.append(pairwise_f1)

print("Best model metrics:", best_model_metrics)
print("Pairwise F1 scores:", pairwise_f1_scores)

# ***Random Forest***

Proposed Model

In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, accuracy_score

# Assuming df_feat_engg is your DataFrame
# Replace df_feat_engg with the name of your DataFrame

# Label encode the 'category' column
le = LabelEncoder()
df_feat_engg['category'] = le.fit_transform(df_feat_engg['category'])

# Drop irrelevant columns
X = df_feat_engg.drop(labels=['collection', 'category', 'description', 'contract_address', 'url', 'blockchain'], axis=1)
y = df_feat_engg['category'].values.astype('int32')

# Impute missing values in X with the mean of each column
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

param = {
    'n_estimators': 100,  # Number of trees in the forest
    'max_depth': None,    # Maximum depth of the tree
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'auto',
    'bootstrap': True,
    'random_state': 42,
}

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics_list = []

for i, (train_index, test_index) in enumerate(fold.split(X_imputed, y)):
    print(i)
    X_train, X_test = X_imputed[train_index], X_imputed[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print(len(X_train), len(X_test), len(y_train), len(y_test))

    model = RandomForestClassifier(**param)
    model.fit(X_train, y_train)

    predict = model.predict_proba(X_test)
    print(predict.shape)

    predicted_class = np.argmax(predict, axis=1)

    is_correct = np.sum(predicted_class == y_test)

    roc_auc = roc_auc_score(y_test, predict, average='macro', multi_class='ovr')
    acc = accuracy_score(y_test, predicted_class)
    f1 = f1_score(y_test, predicted_class, average='micro')
    recall = recall_score(y_test, predicted_class, average='micro')
    precision = precision_score(y_test, predicted_class, average='micro')

    print(f"roc_auc_score    : {roc_auc}")
    print(f"is_correct       : {is_correct}")
    print(f"accuracy         : {acc}")
    print(f"f1_score         : {f1}")
    print(f"recall_score     : {recall}")
    print(f"precision_score  : {precision}")

    metrics_dict = {
        'tot_samples': len(y_test_array),
        'is_correct': is_correct,
        'roc_auc_score': roc_auc,
        'accuracy': acc,
        'f1_score': f1,
        'recall_score': recall,
        'precision_score': precision
    }

    metrics_list.append(metrics_dict)


0
1924 482 1924 482


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


(482, 5)
roc_auc_score    : 0.9793202447797971
is_correct       : 420
accuracy         : 0.8713692946058091
f1_score         : 0.8713692946058091
recall_score     : 0.8713692946058091
precision_score  : 0.8713692946058091


NameError: name 'y_test_array' is not defined

In [ ]:
results_rf = pd.DataFrame(metrics_list)
results_rf

In [ ]:
results_rf.to_csv(f'{path}results/exp-4-rf.csv',index = False)

# ***XGBoost***

Baseline Model

In [16]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, accuracy_score

le = LabelEncoder()
df_feat_engg['category'] = le.fit_transform(df_feat_engg['category'])

print(df_feat_engg.dtypes)

X = df_feat_engg.drop(labels=['collection', 'category','description','contract_address','url','blockchain'], axis=1)
y = df_feat_engg['category'].values.astype('int32')

param = {'objective': 'multi:softmax',
         'num_class': len(np.unique(y)),  # Number of classes
         'learning_rate': 0.1,
         'n_estimators': 5000,
         'min_child_weight': 1,
         'gamma': 0,
         'subsample': 0.8,
         'colsample_bytree': 0.8,
         'eval_metric': 'mlogloss',  # Multi-class log loss
         'verbosity': 0
         }

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics_list = []

for i, (train_index, test_index) in enumerate(fold.split(X, y)):
    print(i)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print(len(X_train), len(X_test), len(y_train), len(y_test))

    model = XGBClassifier(**param)

    model = model.fit(X_train, y_train,
                      early_stopping_rounds=200,
                      eval_set=[(X_test, y_test)],
                      verbose=100)

    predict = model.predict_proba(X_test)
    print(predict.shape)

    # You may need to adjust the following lines based on the output structure of XGBoost
    X_test['predicted_class'] = [np.argmax(i) for i in predict]
    y_test_array = np.array(y_test)

    X_test['is_correct'] = np.where(y_test_array ==  X_test['predicted_class'], 1, 0)

    is_correct = X_test['is_correct'].sum()
    print(f"is_correct      : {X_test['is_correct'].sum()}")

    roc_auc = roc_auc_score(y_test_array, predict, average='macro', multi_class='ovr')
    acc = accuracy_score(y_test_array, X_test['predicted_class'])
    f1 = f1_score(y_test_array, X_test['predicted_class'], average='micro')
    recall = recall_score(y_test_array, X_test['predicted_class'], average='micro')
    precision = precision_score(y_test_array, X_test['predicted_class'], average='micro')

    print(f"roc_auc_score    : {roc_auc}")
    print(f"accuracy         : {acc}")
    print(f"f1_score         : {f1}")
    print(f"recall_score     : {recall}")
    print(f"precision_score  : {precision}")

    metrics_dict = {
        'tot_samples': len(y_test_array),
        'is_correct': is_correct,
        'roc_auc_score': roc_auc,
        'accuracy': acc,
        'f1_score': f1,
        'recall_score': recall,
        'precision_score': precision
    }

    metrics_list.append(metrics_dict)


AaANwg8                                     float64
AcoraidaMonicaWantsToKeepALogOfTheWinner    float64
Activate                                    float64
AddFish                                     float64
AddModerator                                float64
                                             ...   
url                                          object
collection                                   object
description                                  object
category                                      int64
blockchain                                   object
Length: 2257, dtype: object
0
1924 482 1924 482


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.44165
[100]	validation_0-mlogloss:0.27370
[200]	validation_0-mlogloss:0.27720
[300]	validation_0-mlogloss:0.28702
[324]	validation_0-mlogloss:0.28960
(482, 5)
is_correct      : 418
roc_auc_score    : 0.9890242097803469
accuracy         : 0.8672199170124482
f1_score         : 0.8672199170124482
recall_score     : 0.8672199170124482
precision_score  : 0.8672199170124482
1
1925 481 1925 481


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.43504
[100]	validation_0-mlogloss:0.21488
[200]	validation_0-mlogloss:0.21804
[300]	validation_0-mlogloss:0.22373
[350]	validation_0-mlogloss:0.22671
(481, 5)
is_correct      : 437
roc_auc_score    : 0.9888231291633514
accuracy         : 0.9085239085239085
f1_score         : 0.9085239085239085
recall_score     : 0.9085239085239085
precision_score  : 0.9085239085239085
2
1925 481 1925 481


...

[0]	validation_0-mlogloss:1.44036
[100]	validation_0-mlogloss:0.27692
[200]	validation_0-mlogloss:0.29016
[280]	validation_0-mlogloss:0.30038
(481, 5)
is_correct      : 423
roc_auc_score    : 0.9862605173751555
accuracy         : 0.8794178794178794
f1_score         : 0.8794178794178794
recall_score     : 0.8794178794178794
precision_score  : 0.8794178794178794
3
1925 481 1925 481


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.44084
[100]	validation_0-mlogloss:0.25419
[200]	validation_0-mlogloss:0.25452
[300]	validation_0-mlogloss:0.25690
[333]	validation_0-mlogloss:0.25814
(481, 5)
is_correct      : 431
roc_auc_score    : 0.9904341502241717
accuracy         : 0.896049896049896
f1_score         : 0.896049896049896
recall_score     : 0.896049896049896
precision_score  : 0.896049896049896
4
1925 481 1925 481


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.43736
[100]	validation_0-mlogloss:0.29501
[200]	validation_0-mlogloss:0.32286
[270]	validation_0-mlogloss:0.33319
(481, 5)
is_correct      : 417
roc_auc_score    : 0.9869778489821291
accuracy         : 0.8669438669438669
f1_score         : 0.866943866943867
recall_score     : 0.8669438669438669
precision_score  : 0.8669438669438669


In [17]:
results_xgboost = pd.DataFrame(metrics_list)
results_xgboost

,tot_samples,is_correct,roc_auc_score,accuracy,f1_score,recall_score,precision_score
0,482,418,0.989024,0.867220,0.867220,0.867220,0.867220
1,481,437,0.988823,0.908524,0.908524,0.908524,0.908524
2,481,423,0.986261,0.879418,0.879418,0.879418,0.879418
3,481,431,0.990434,0.896050,0.896050,0.896050,0.896050
4,481,417,0.986978,0.866944,0.866944,0.866944,0.866944


In [18]:
results_xgboost.to_csv(f'{path}results/exp-4-xgboost.csv',index = False)

In [19]:
data = {
        'actual_label': X_test['predicted_class'].values,
        'prediction_label_argmax': y_test_array,
}
df = pd.DataFrame(data)
df

,actual_label,prediction_label_argmax
0,3,2
1,3,3
2,1,1
3,2,2
4,3,3
...,...,...
476,3,3
477,1,1
478,3,3
479,3,3


In [20]:
df.to_csv(f'{path}results/exp-4-xgboost-results.csv',index = False)

In [22]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(df['actual_label'],df['prediction_label_argmax'])
confusion_matrix

array([[111,   7,   0,   0,   0],
       [  6, 113,   0,   0,   0],
       [  1,   0,  52,   0,   7],
       [  2,   0,  11,  81,  15],
       [  0,   0,  10,   5,  60]])

# ***Results***

In [ ]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import f1_score

le = LabelEncoder()
df_feat_engg['category'] = le.fit_transform(df_feat_engg['category'])

X = df_feat_engg.drop(labels=['collection', 'category','description','contract_address','url','blockchain'], axis=1)
y = df_feat_engg['category'].values.astype('int32')

param = {'objective': 'multi:softmax',
         'num_class': len(np.unique(y)),  # Number of classes
         'learning_rate': 0.1,
         'n_estimators': 5000,
         'min_child_weight': 1,
         'gamma': 0,
         'subsample': 0.8,
         'colsample_bytree': 0.8,
         'eval_metric': 'mlogloss',  # Multi-class log loss
         'verbosity': 0
         }

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics_list = []

for i, (train_index, test_index) in enumerate(fold.split(X, y)):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = XGBClassifier(**param)

    model = model.fit(X_train, y_train,
                      early_stopping_rounds=200,
                      eval_set=[(X_test, y_test)],
                      verbose=100)

    predict = model.predict_proba(X_test)

    # You may need to adjust the following lines based on the output structure of XGBoost
    X_test['predicted_class'] = [np.argmax(i) for i in predict]
    y_test_array = np.array(y_test)

    # Calculate F1 score for each category separately
    f1_scores = f1_score(y_test_array, X_test['predicted_class'], average=None)

    print(f"F1 Scores for each category: {f1_scores}")

    metrics_dict = {
        'f1_scores': f1_scores,
    }

    metrics_list.append(metrics_dict)

    break

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.44165
[100]	validation_0-mlogloss:0.27370
[200]	validation_0-mlogloss:0.27720
[300]	validation_0-mlogloss:0.28702
[325]	validation_0-mlogloss:0.28957
F1 Scores for each category: [0.91525424 0.93442623 0.7388535  0.86585366 0.82208589]


In [ ]:
metrics_list

[{'f1_scores': array([0.91525424, 0.93442623, 0.7388535 , 0.86585366, 0.82208589])}]